In [12]:
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, date, timedelta
from pathlib import Path
today = str(datetime.now())[:10]
import pickle
import matplotlib
import matplotlib as mpl
matplotlib.use('Agg')
from matplotlib import style
from matplotlib import pyplot as plt
plt.style.use('ggplot')
sm, med, lg = 10, 15, 20
plt.rc('font', size = sm)         # controls default text sizes
plt.rc('axes', titlesize = med)   # fontsize of the axes title
plt.rc('axes', labelsize = med)   # fontsize of the x & y labels
plt.rc('xtick', labelsize = sm)   # fontsize of the tick labels
plt.rc('ytick', labelsize = sm)   # fontsize of the tick labels
plt.rc('legend', fontsize = sm)   # legend fontsize
plt.rc('figure', titlesize = lg)  # fontsize of the figure title
plt.rc('axes', linewidth=2)       # linewidth of plot lines
plt.rcParams['figure.figsize'] = [18, 10]
plt.rcParams['figure.dpi'] = 150

import pandas as pd
import numpy as np
import itertools
import streamlit as st
st.set_option('deprecation.showPyplotGlobalUse', False)
import yfinance as yf
import yahoo_fin.stock_info as si
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow import keras

In [13]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
    tf.config.experimental.set_synchronous_execution(enable=True)
    tf.config.experimental.enable_mlir_bridge
    tf.config.experimental.enable_tensor_float_32_execution(enabled=True)
    tf.config.threading.get_inter_op_parallelism_threads()
    tf.config.threading.set_inter_op_parallelism_threads(0)
else:
    print('Using CPU')

Using CPU


In [14]:
def clean(listA):
    lst = list(set(listA))
    lst.sort()
    return lst

saveTickers = Path('/home/gordon/gdp/code/portfolio/Forecasting_For_Friends/tickers')

chuck_merged_ticker_lst = pd.read_pickle(saveTickers / f'chuck_merged_ticker_lst.pkl')
watch_merged_ticker_lst = pd.read_pickle(saveTickers / f'watch_merged_ticker_lst.pkl')

dow = pd.read_pickle(saveTickers / f'dow_ticker_lst.pkl')
sp100 = pd.read_pickle(saveTickers / f'sp100_ticker_lst.pkl')
sp500 = pd.read_pickle(saveTickers / f'sp500_ticker_lst.pkl')

day_gainers = list(si.get_day_gainers()['Symbol'])
day_losers = list(si.get_day_losers()['Symbol'])
day_most_active = list(si.get_day_most_active()['Symbol'])
undervalued_large_caps = pd.read_pickle(saveTickers / f"undervalued_large_caps_ticker_lst.pkl")

fool_composite = [
    'LMND','ZM','TTD','PINS','TEAM','SAM','DIS','ASML','ECL','NYT',
    'LRCX','NTDOY','PYPL','AMZN','ABNB','ATVI','ZM','SKLZ','SHOP', 'STAA',
    'LULU','WING', 'ETSY','BL','RDFN','LOGI','EQIX','U','RGEN','CHGG',
    'PINS','FUBO','W','MRNA','AXON','SNBR','TDOC','GDRX','PLNT','PLNT',
    'GDRX','RDFN','PINS','LULU','AVAV','FSLY','AXON','BL','ZEN','DDOG',
    'NEE','CRM','TEAM','ZG','Z','TWLO','RMD','STAA','WING','ETSY',
    'LOGI','EQIX','U','RGEN','CHGG','FUBO','LO','MRNA','SNBR','TDOC',
    'AAPL','ASML','BLDP','BA','CRLBF','GWPH','NVDA','PLTR','SNOW','SIVB',
    'TSLA','HRVSF','ARKG','ARKK','GBTC','ECL','NNOX','OKTA','CRM','SOHU',
    'FVRR','LTCN','OROCF','ETCG','APHA','BILI','CLLS','COUP','CUE','NYT',
    'RIOT','SE','SQ','TECK'    
]
oxford_composite = [
  'BZH','CROX','HA','HAS','PFGC','POOL','GDOT','HUYA','GRUB','FSLR','SPWR',
  'GS','BYND','PFGC','VRA','NLOK','NET','BYND','YETI','CURLF','ALB','WMT',
  'DG','BCO','NFE','UBER','RUN','BABA','FAST','CRLBF','LUN-T','TRIP','FCEL',
  'ALB','BABA','FAST','FCEL','LUN-T','YETI','BCO','DG','DLR','GRMN','WMT',
  'LSCC','MU','NVDA','TRIP','UBER','CRLBF','CURLF','IIPR','CVS','XLE','IBM',
  'ORCL','VZ','DEM','WIX','ZEN','VWEHX, VSMAX','VIPSX','VFSTX','VEMAX',
  'VTSAX','VPADX','VGSLX','VEUSX','GDX','GKOS','MRVL','NEO','NVCR','PFPT',
  'TDF','PSHZF','MKL','IEP','EQR','EMF','BRK-B','AIG','F','PFE','V','WMT',
  'JLL','MCK','RDS-B','BUD','DAL','RIOT','MRNA','AMKBY','BRPHF','ZEST',
  'WIX','PSHZF','ZEN','IBM','VZ','XLE','ORCL','NEO','LBTYA','DRNA','EA',
  'EGHT','BZUN','TTWO','EBAY','FTNT','SAIL','BABA','SAM','COLM','DPZ',
  'EXPE','NUVA','EA','HSY','HAS','NFLX','SIX'
]

dow_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_DOW.pkl')
sp100_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_SP100.pkl')
sp500_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_SP500.pkl')

day_gainers_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_day_gainers.pkl')
day_losers_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_day_losers.pkl')
day_most_active_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_day_most_active.pkl')
undervalued_large_caps_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_undervalued_large_caps_ticker_lst.pkl')

fool_composite_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_fool_composite.pkl')
oxford_composite_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_oxford_composite.pkl')
watch_lst_bulk_analyst_buys1 = pd.read_pickle(saveTickers / f'recommendations_watch_lst_bulk.pkl')

In [16]:
index_ticker_lists_A = [
  dow, sp100, sp500, 
  chuck_merged_ticker_lst, watch_merged_ticker_lst, fool_composite, oxford_composite,
  day_gainers, day_losers, day_most_active, undervalued_large_caps,

  dow_analyst_buys1, sp100_analyst_buys1, sp500_analyst_buys1, 
#   chuck_merged_analyst_buys1, watch_merged_analyst_buys1, 
  fool_composite_analyst_buys1, oxford_composite_analyst_buys1,  
  day_gainers_analyst_buys1, day_losers_analyst_buys1, day_most_active_analyst_buys1, undervalued_large_caps_analyst_buys1
]

index_ticker_lists_B = [
  'dow', 'sp100', 'sp500', 
  'chuck_merged_ticker_lst', 'watch_merged_ticker_lst', 'fool_composite', 'oxford_composite',
  'day_gainers', 'day_losers', 'day_most_active', 'undervalued_large_caps',
  
  'dow_analyst_buys1', 'sp100_analyst_buys1', 'sp500_analyst_buys1', 
#   'chuck_merged_analyst_buys1', 'watch_merged_analyst_buys1',
  'fool_composite_analyst_buys1', 'oxford_composite_analyst_buys1'
  'day_gainers_analyst_buys1', 'day_losers_analyst_buys1', 'day_most_active_analyst_buys1', 'undervalued_large_caps_analyst_buys1'
]

for r in range(len(index_ticker_lists_A)):
  index_ticker_lists_A[r] = clean(index_ticker_lists_A[r])